In [ ]:
!pip install boto3 -q

     |████████████████████████████████| 132 kB 26.9 MB/s 
     |████████████████████████████████| 9.0 MB 64.2 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 139 kB 72.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

In [ ]:
import boto3
s3 = boto3.resource('s3') 

def download_s3_folder(bucket_name, s3_folder, local_dir=None):
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        bucket.download_file(obj.key, target)

In [ ]:
download_s3_folder(bucket_name="jeongmin-mask-detection-test-dataset", s3_folder="images")

In [ ]:
download_s3_folder(bucket_name="jeongmin-mask-detection-test-dataset", s3_folder="annotations")

In [ ]:
import os

import xml.etree.ElementTree as ET

annotations_path = "/content/annotations"
yolo_annotations_path = "/content/yolo/labels"
names_to_idx_dict = {
    "mask_weared_incorrect": 0,
    "with_mask": 1,
    "without_mask": 2
}

annos = os.listdir(annotations_path)
os.makedirs(yolo_annotations_path, exist_ok=True)

def get_info(ann_root):
    filename = ann_root.findtext('filename')
    assert filename is not None
    img_name = os.path.basename(filename)

    size = ann_root.find('size')
    width = float(size.findtext('width'))
    height = float(size.findtext('height'))

    objs = ann_root.findall("object")
    classes = []
    bboxes = []
    for obj in objs:
        classes.append(obj.findtext("name"))

        bbox = obj.find("bndbox")
        xmin = int(bbox.findtext("xmin"))
        ymin = int(bbox.findtext("ymin"))
        xmax = int(bbox.findtext("xmax"))
        ymax = int(bbox.findtext("ymax"))
        bboxes.append([xmin, ymin, xmax, ymax])

    image_info = {
        'file_name': filename,
        'height': height,
        'width': width,
        'classes': classes,
        'bboxes': bboxes
    }
    return image_info

def make_yolo_format(image_info):
    img_width = image_info["width"]
    img_height = image_info["height"]

    results_lines = []
    for i in range(len(image_info["classes"])):
        class_str = image_info["classes"][i]

        x1 = float(image_info["bboxes"][i][0])
        y1 = float(image_info["bboxes"][i][1])
        x2 = float(image_info["bboxes"][i][2])
        y2 = float(image_info["bboxes"][i][3])
        
        intx1 = int(x1)
        inty1 = int(y1)
        intx2 = int(x2)
        inty2 = int(y2)

        bbox_center_x = float( (x1 + (x2 - x1) / 2.0) / img_width)
        bbox_center_y = float( (y1 + (y2 - y1) / 2.0) / img_height)
        bbox_width = float((x2 - x1) / img_width)
        bbox_height = float((y2 - y1) / img_height)

        line_to_write = str(names_to_idx_dict[class_str]) + ' ' + str(bbox_center_x)+ ' ' + str(bbox_center_y)+ ' ' + str(bbox_width)+ ' ' + str(bbox_height) +'\n'
    
        results_lines.append(line_to_write)
    return results_lines

for anno_path in annos:
    ann_tree = ET.parse(os.path.join(annotations_path, anno_path))
    ann_root = ann_tree.getroot()

    img_info = get_info(ann_root)
    yolo_lines = make_yolo_format(img_info)

    with open(f"{yolo_annotations_path}/{img_info['file_name'].split('.')[0]}.txt", "w") as file:
        file.writelines(yolo_lines)

In [ ]:
import os
import shutil
import random

yolo_img_dir = "images"
yolo_label_dir = "yolo/labels"

target_dataset_dir = "coco"

os.makedirs(f"{target_dataset_dir}/images/train", exist_ok=True)
os.makedirs(f"{target_dataset_dir}/images/val", exist_ok=True)
os.makedirs(f"{target_dataset_dir}/labels/train", exist_ok=True)
os.makedirs(f"{target_dataset_dir}/labels/val", exist_ok=True)

data_list = os.listdir(yolo_label_dir)
for data in data_list:
    data_name = data.split(".")[0]

    if random.uniform(0, 1) > 0.2:
        shutil.copyfile(f"{yolo_img_dir}/{data_name}.png", f"{target_dataset_dir}/images/train/{data_name}.png")
        shutil.copyfile(f"{yolo_label_dir}/{data_name}.txt", f"{target_dataset_dir}/labels/train/{data_name}.txt")

    else:
        shutil.copyfile(f"{yolo_img_dir}/{data_name}.png", f"{target_dataset_dir}/images/val/{data_name}.png")
        shutil.copyfile(f"{yolo_label_dir}/{data_name}.txt", f"{target_dataset_dir}/labels/val/{data_name}.txt")

In [ ]:
!python /content/yolov7/train.py \
--workers 8 \
--device 0 \
--batch-size 16 \
--data /content/yolov7/data/coco.yaml \
--img 640 640 \
--cfg /content/yolov7/cfg/training/yolov7.yaml \
--weights /content/yolov7.pt \
--name yolov7 \
--hyp /content/yolov7/data/hyp.scratch.p5.yaml


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
YOLOR 🚀 v0.1-20-gf5d50fa torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='/content/yolov7/cfg/training/yolov7.yaml', data='/content/yolov7/data/coco.yaml', device='0', entity=None, epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='/content/yolov7/data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov7', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov73', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='/content/yolov7.pt', workers=8, world

In [ ]:
!git clone https://github.com/silverstar0727/yolov7-finetune-for-mask-images

Cloning into 'yolov7-finetune-for-mask-images'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 130 (delta 47), reused 124 (delta 41), pack-reused 0
Receiving objects: 100% (130/130), 872.23 KiB | 29.07 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:
import boto3
s3 = boto3.resource('s3') 
bucket = s3.Bucket("jeongmin-mlflow-bucket")

bucket.download_file("best.pt", "best.pt")

In [ ]:
!python /content/yolov7-finetune-for-mask-images/yolov7/detect.py --weights best.pt --conf 0.25 --img-size 640 --source maksssksksss3.png

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='maksssksksss3.png', update=False, view_img=False, weights=['best.pt'])
YOLOR 🚀 3b0ac39 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 314 layers, 36492560 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered intern